# imports

In [ ]:
# beomi/kcbert-base
!pip install transformers
!pip install torchmetrics
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/QIA-Hackathon2023")

import pandas as pd
import numpy as np


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# create dataset

In [ ]:
train=pd.read_csv('df_train_raw.csv', index_col=0)
test=pd.read_csv('df_valid_raw.csv', index_col=0)

df_questions = pd.read_excel("Question.xlsx")

df_submission_test = pd.read_csv("hackathon_test_for_user.csv", encoding="cp949")

In [ ]:
# MBTI mapping 
MBTI_mapping = [
    {'I': 0, 'E': 1},
    {'S': 0, 'N': 1},
    {'T': 0, 'F': 1},
    {'J': 0, 'P': 1},
]
# 0 - ISTJ; 1 - ENFP 


In [ ]:
def create_promt (row):
    p = f"젠더: {row.Gender}.\n 나이: {row.Age}.\n 질문: {df_questions.Question.loc[row.Q_number-1]}.\n 정답: {row.Answer}.\n"
    return p



In [ ]:
train["prompt"] = train.apply(create_promt, axis=1)
test["prompt"] = test.apply(create_promt, axis=1)
df_submission_test["prompt"] = df_submission_test.apply(create_promt, axis=1)


In [ ]:
def create_label_mbti (mbti):
  return np.array([MBTI_mapping[i][letter] for i, letter in enumerate(mbti)])

print ("INFP: ", create_label_mbti("INFP"))

def create_label_user_ID (user_ID):
  b = np.zeros([240])
  b[user_ID-1]=1
  return b

# test["label_mbti"] = test.MBTI.map( create_label_mbti)
# train["label_mbti"] = train.MBTI.map( create_label_mbti )

# train["label_userID"] = train.User_ID.map( create_label_user_ID)
# test["label_userID"] = test.User_ID.map( create_label_user_ID)

INFP:  [0 1 1 1]


In [ ]:
# IMPORTANT: ID's the model will predict are from 0 to 239
def create_label (row):
  mbti = create_label_mbti(row.MBTI)
  id = row.User_ID-1 
  return np.concatenate( [mbti, np.array([id]) ] , axis = 0)


In [ ]:
train["label"] = train.apply(create_label, axis=1)
test["label"] = test.apply(create_label, axis=1)



In [ ]:
train.tail()

,Data_ID,User_ID,Gender,Age,MBTI,Q_number,Answer,prompt,label
11507,14376,240,0,40,ISTJ,36,<아니다> 저는 즐거운 파티나 행사로 일주일 피로를 푸는 편이 아닙니다. 이유는 그...,젠더: 0.\n 나이: 40.\n 질문: 혼자만의 시간을 보내기보다는 즐거운 파티와...,"[0, 0, 0, 0, 239]"
11508,14377,240,0,40,ISTJ,37,<중립> 저는 미술관 가는 일을 좋아하지 않습니다. 이유는 미술 작품에 관심이 많이...,젠더: 0.\n 나이: 40.\n 질문: 미술관에 가는 일을 좋아하나요? 꼭 미술관...,"[0, 0, 0, 0, 239]"
11509,14378,240,0,40,ISTJ,38,<그렇다> 저는 다른 사람의 감정을 이해하기 힘들 때가 많습니다. 이유는 각자의 살...,젠더: 0.\n 나이: 40.\n 질문: 다른 사람의 감정을 이해하기 힘들 때가 많...,"[0, 0, 0, 0, 239]"
11510,14379,240,0,40,ISTJ,39,<아니다> 저는 매일 할 일을 계획하지 않습니다. 이유는 매일매일보다는 크게 한 건...,젠더: 0.\n 나이: 40.\n 질문: 매일 할 일을 계획하는 일을 좋아하나요? ...,"[0, 0, 0, 0, 239]"
11511,14380,240,0,40,ISTJ,40,<아니다> 저는 불안함을 많이 느낍니다. 이유는 모든 것이 확실히 되야지만 마음이 ...,젠더: 0.\n 나이: 40.\n 질문: 불안함을 느낄 때가 거의 없나요? 타고난 ...,"[0, 0, 0, 0, 239]"


# def model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

class KoreanTextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long),
        }


In [ ]:
from torch.nn import functional as F
from transformers import AutoModel
# model = AutoModelForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels=2)

class MBTI_UserID_Model(torch.nn.Module):

    def __init__(self, large = False, dropout_per = 0.1, total_Users = 240):
        super(MBTI_UserID_Model, self).__init__()        
        
        if large: 
          model_features = 1024
          self.base_model = AutoModel.from_pretrained("beomi/kcbert-large", )
        
        else: 
          model_features = 768
          self.base_model = AutoModel.from_pretrained("beomi/kcbert-base", )

        self.linear1 = torch.nn.Sequential(torch.nn.Dropout(p=dropout_per), torch.nn.Linear(model_features, 1))
        self.linear2 = torch.nn.Sequential(torch.nn.Dropout(p=dropout_per), torch.nn.Linear(model_features, 1))
        self.linear3 = torch.nn.Sequential(torch.nn.Dropout(p=dropout_per), torch.nn.Linear(model_features, 1))
        self.linear4 = torch.nn.Sequential(torch.nn.Dropout(p=dropout_per), torch.nn.Linear(model_features, 1))
        self.linear_users = torch.nn.Sequential(torch.nn.Dropout(p=dropout_per), torch.nn.Linear(model_features, total_Users))
        

    def forward(self, x, attention_mask):
        x = self.base_model (x, attention_mask=attention_mask).pooler_output

        return self.linear1(x), self.linear2(x), self.linear3(x), self.linear4(x), self.linear_users(x)



# set up training

In [ ]:
test.head()


,Data_ID,User_ID,Gender,Age,MBTI,Q_number,Answer,prompt,label
40,41,1,1,30,INFP,41,<그렇다> 말을 잘 못하기 때문에 말실수를 할까 봐 피하고 싶습니다. 문자로 하는 ...,젠더: 1.\n 나이: 30.\n 질문: 전화 통화를 거는 일은 가능한 피하고 싶나...,"[0, 1, 1, 1, 0]"
41,42,1,1,30,INFP,42,<그렇다> 아주 많은 시간을 할애하는 편은 아니지만 노력하는 편입니다. 너무 다른 ...,젠더: 1.\n 나이: 30.\n 질문: 자신의 의견과 매우 다른 의견을 이해하기 ...,"[0, 1, 1, 1, 0]"
42,43,1,1,30,INFP,43,<아니다> 대부분 친구가 먼저 만나자고 하면 가는 편입니다. 저는 연락을 대체로 먼...,젠더: 1.\n 나이: 30.\n 질문: 친구에게 먼저 만나자고 연락하는 편인가요?...,"[0, 1, 1, 1, 0]"
43,44,1,1,30,INFP,44,<그렇다> 차질이 생기면 그 부분만 다르게 하고 원래 계획으로 돌아가기 위해 노력합...,젠더: 1.\n 나이: 30.\n 질문: 계획에 차질이 생기면 최대한 빨리 계획으로...,"[0, 1, 1, 1, 0]"
44,45,1,1,30,INFP,45,<그렇다> 굉장히 오래전의 실수라 이젠 바뀔 수 없음에도 종종 후회합니다. 수능 끝...,젠더: 1.\n 나이: 30.\n 질문: 오래전의 실수를 후회할 때가 많나요? 요즘...,"[0, 1, 1, 1, 0]"


In [ ]:
use_train_val_combined = False
train_val_combined = pd.concat([train, test])
train_val_combined.tail()


,Data_ID,User_ID,Gender,Age,MBTI,Q_number,Answer,prompt,label
11515,14384,240,0,40,ISTJ,44,<그렇다> 저는 계획에 차질이 생기면 돌아가기 위해 노력을 합니다. 이유는 그 계획...,젠더: 0.\n 나이: 40.\n 질문: 계획에 차질이 생기면 최대한 빨리 계획으로...,"[0, 0, 0, 0, 239]"
11516,14385,240,0,40,ISTJ,45,<그렇다> 저는 예전의 실수를 후회할 때가 많습니다. 이유는 그만큼 나태하게 산 적...,젠더: 0.\n 나이: 40.\n 질문: 오래전의 실수를 후회할 때가 많나요? 요즘...,"[0, 0, 0, 0, 239]"
11517,14386,240,0,40,ISTJ,46,<아니다> 저는 인간의 존재와 삶의 이유에 대해 깊이 생각하지 않습니다. 이유는 이...,젠더: 0.\n 나이: 40.\n 질문: 인간의 존재와 삶의 이유에 대해 깊이 생각...,"[0, 0, 0, 0, 239]"
11518,14387,240,0,40,ISTJ,47,<아니다> 저는 감정에 휘둘리는 편이 아닙니다. 이유는 감정을 감추고 밖으로 표현하...,젠더: 0.\n 나이: 40.\n 질문: 감정을 조절하기보다는 감정에 휘둘리는 편인...,"[0, 0, 0, 0, 239]"
11519,14388,240,0,40,ISTJ,48,<아니다> 저는 상대방 잘못일 때 상대방의 체면을 살려주기 위해 노력하지 않습니다....,젠더: 0.\n 나이: 40.\n 질문: 상대방의 잘못이라는 것이 명백할 때도 상대...,"[0, 0, 0, 0, 239]"


In [ ]:
train_model_type_large = False
train_model_type_large = True

if train_model_type_large: 
  tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-large")
else: 
  tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")

if use_train_val_combined: 
  train_texts=list(train_val_combined['prompt'])
  train_labels= list(train_val_combined['label'])
else: 
  train_texts=list(train['prompt'])
  train_labels= list(train['label'])
# train_labels= [list(train['label_mbti']), list(train['label_userID'])]

test_texts=list(test['prompt'])
test_labels= list(test['label'])
# test_labels= [list(test['label_mbti']), list(test['label_userID'])]

batch_size = 32

max_length = 128

train_dataset = KoreanTextDataset(train_texts, train_labels, tokenizer, max_length=max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = KoreanTextDataset(test_texts, test_labels, tokenizer, max_length=max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [ ]:
# model = MBTI_UserID_Model(large = train_model_type_large, dropout_per = 0.3 )

model = MBTI_UserID_Model(large = train_model_type_large, dropout_per = 0.5 )

Some weights of the model checkpoint at beomi/kcbert-large were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# model eval

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print (device)

cuda


In [ ]:
from torchmetrics.classification import BinaryAUROC
from sklearn.metrics import roc_auc_score, accuracy_score

loss_cross_en_userID = torch.nn.CrossEntropyLoss()
user_id_loss_coef = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# eval loop:
def model_eval(): 
  model.eval()
  predictions = [[] for i in range(4)]
  true_labels = [[] for i in range(4)]

  predictions_usrID = []
  true_labels_usrID = []

  with torch.no_grad():
      total_human_loss = 0
      total_mbti_loss = 0
      iter = 0
      for batch in test_loader:

          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          outputs = model(input_ids, attention_mask=attention_mask)
          labels = batch['label'].to(device)

          
          loss = 0
          for i in range(4): 
              loss += F.binary_cross_entropy_with_logits( outputs[i][:, 0], labels[:,i].type(torch.FloatTensor).to(device) ) 
          
          # user ID class loss: 
          humna_class_loss = user_id_loss_coef*loss_cross_en_userID(outputs[4], labels[:, 4])
          
          total_mbti_loss+=loss
          total_human_loss+=humna_class_loss
          
          iter += 1

          labels.cpu().numpy()
          for i in range(4):
            
            logits = outputs[i]
            prob = F.sigmoid(logits).cpu().numpy() 
            predictions[i].append(prob)
            
            label = batch['label'][:, i]
            true_labels[i].append(label)
          
          prob = outputs[4].softmax(dim=1).cpu().numpy() 
          predictions_usrID.append(prob)
          label = batch['label'][:, 4]
          true_labels_usrID.append(label)

  for i in range(4):
      score = roc_auc_score(np.concatenate(true_labels[i]), np.concatenate(predictions[i]))
      print (f"score for {i}: {score}. ")
  accuracy_score_user = accuracy_score (  np.concatenate(true_labels_usrID), np.concatenate(predictions_usrID).argmax(axis = 1) )

  total_human_loss = total_human_loss/iter
  total_mbti_loss = total_mbti_loss/iter

  print ("accuracy:", accuracy_score_user)
  print ("losses:", float(total_human_loss.cpu().numpy()), float(total_mbti_loss.cpu().numpy()))


# training loop

In [ ]:
# optimizer = torch.optim.Adam(model.parameters(), lr=2e-6)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
lr = 1e-5
# lr = 2e-6

epochs = 15
loss_cross_en_userID = torch.nn.CrossEntropyLoss()
user_id_loss_coef = 1

from torchmetrics.classification import BinaryAUROC
from sklearn.metrics import roc_auc_score

def sigmoid (x):
  return 1/(1 + np.exp ( -x ))


In [ ]:
epochs = 5

model_name = "large_3_"
# Training loop
for epoch in range(epochs):
    if epoch == 0: 
      optimizer = torch.optim.Adam(model.parameters(), lr=2e-6)
    else: 
      optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.train()
    total_loss = 0

    total_human_loss = 0
    total_mbti_loss = 0

    predictions = [[] for i in range(4)]
    true_labels = [[] for i in range(4)]

    predictions_usrID = []
    true_labels_usrID = []


    iter = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, )

        loss = 0
        for i in range(4): 
            loss += F.binary_cross_entropy_with_logits( outputs[i][:, 0].to(device), labels[:,i].type(torch.FloatTensor).to(device)) 

        total_mbti_loss+=loss
        # user ID class loss: 
        humna_class_loss = user_id_loss_coef*loss_cross_en_userID(outputs[4], labels[:, 4])
        total_human_loss+=humna_class_loss
        loss+= humna_class_loss

        loss.backward()
        optimizer.step()
        total_loss += loss
        iter += 1


        # labels.cpu().numpy()
        for i in range(4):
            
            logits = outputs[i]
            prob = F.sigmoid(logits).detach().cpu().numpy() 
            predictions[i].append(prob)
            
            label = batch['label'][:, i]
            true_labels[i].append(label)
          
            prob = outputs[4].softmax(dim=1).detach().cpu().numpy() 
            predictions_usrID.append(prob)
            label = batch['label'][:, 4]
            true_labels_usrID.append(label)
        
    
    print()
    print (f"Train results epoh {epoch}:")
    for i in range(4):
      score = roc_auc_score(np.concatenate(true_labels[i]), np.concatenate(predictions[i]))
      print (f"score for {i}: {score}. ")
    accuracy_score_user = accuracy_score (  np.concatenate(true_labels_usrID), np.concatenate(predictions_usrID).argmax(axis = 1) )
    print (f"accuracy_train: {accuracy_score_user}. ")

    
    print(f"Done epoch: {epoch}. Train loss = {total_loss/iter}")
    total_human_loss = total_human_loss/iter
    total_mbti_loss = total_mbti_loss/iter

    print ("losses:", float(total_human_loss.detach().cpu().numpy()), float(total_mbti_loss.detach().cpu().numpy()))

    print()
    print (f"Eval results at epoh {epoch}: ")
    model_eval()



Train results epoh 0:
score for 0: 0.9936313368055556. 
score for 1: 0.9825974826388889. 
score for 2: 0.9831856770833334. 
score for 3: 0.9786948350694443. 
accuracy_train: 0.6323958333333334. 
Done epoch: 0. Train loss = 2.5610480308532715
losses: 1.7966634035110474 0.7643834948539734

Eval results at epoh 0: 
score for 0: 0.7853342013888889. 
score for 1: 0.7417046440972221. 
score for 2: 0.7561393229166666. 
score for 3: 0.7619097222222222. 
accuracy: 0.2869791666666667
losses: 2.732877731323242 2.833678722381592

Train results epoh 1:
score for 0: 0.9940105685763889. 
score for 1: 0.9792828559027779. 
score for 2: 0.9813782769097223. 
score for 3: 0.9741616319444445. 
accuracy_train: 0.6353125. 
Done epoch: 1. Train loss = 2.528186082839966
losses: 1.765736699104309 0.7624490261077881

Eval results at epoh 1: 
score for 0: 0.7796272786458333. 
score for 1: 0.7361414930555555. 
score for 2: 0.7565879991319444. 
score for 3: 0.7510243055555557. 
accuracy: 0.2989583333333333
losses:

In [ ]:
PATH = "your path"
torch.save(model.state_dict(), PATH)


# create submission file

In [ ]:
sub_file_prefix = "small_2"
sub_file_prefix = model_name + "_add_"
print (sub_file_prefix)
"hackathon_test_for_user.csv"
df_submission_test


large_3__add_


,Gender,Age,Q_number,Answer,prompt,I/E,S/N,T/F,J/P,userIDs predictions
0,0,30,59,<아니다> 저는 모든 일은 정해진 시간을 지켜서 해야 된다고 생각되어서 마감 기한을...,젠더: 0.\n 나이: 30.\n 질문: 마감 기한을 지키기가 힘든가요? 경험을 이...,0.075223,0.036331,0.966207,0.016445,"[9.7099524e-05, 0.0015523263, 0.00053531304, 0..."
1,1,40,53,<중립> 저는 조용하고 사적인 장소도 좋아하고 사람들로 붐비고 떠들썩한 장소도 좋아...,젠더: 1.\n 나이: 40.\n 질문: 조용하고 사적인 장소보다는 사람들로 붐비고...,0.638643,0.008231,0.976241,0.943661,"[0.0011357091, 0.003327079, 0.0019437977, 0.00..."
2,1,40,56,<그렇다> 저는 규칙을 잘 지키고 매뉴얼 대로 일하는 사람입니다. 그래서 데이터 라...,젠더: 1.\n 나이: 40.\n 질문: 단계를 건너뛰는 일 없이 절차대로 일을 완...,0.566348,0.090936,0.928147,0.130266,"[0.0006127075, 0.0018116662, 0.0009952545, 0.0..."
3,1,40,60,<그렇다> 저는 항상 긍정적인 사고방식을 가지고 살려고 노력하고 있습니다. 이유는 ...,젠더: 1.\n 나이: 40.\n 질문: 일이 원하는 대로 진행될 것이라는 자신감이...,0.964644,0.074085,0.977200,0.110636,"[0.00047402713, 0.000898123, 0.0008986662, 0.0..."
4,1,30,51,<중립> 혼자서 일하는 것도 좋고 함께 일하는 것도 모두 좋은데 같이 의논하는 일도...,젠더: 1.\n 나이: 30.\n 질문: 대부분의 시간을 혼자서 일할 수 있는 직업...,0.012456,0.035689,0.425572,0.968727,"[0.005169144, 0.0013867167, 0.002398192, 0.000..."
...,...,...,...,...,...,...,...,...,...,...
2875,1,20,59,<아니다> 마감 기한은 웬만해서는 지키려고 하는 편이에요. 거의 대부분 지각하지 않아요.,젠더: 1.\n 나이: 20.\n 질문: 마감 기한을 지키기가 힘든가요? 경험을 이...,0.967255,0.018763,0.067738,0.976374,"[0.00061842083, 0.001009285, 0.001005655, 0.00..."
2876,0,20,54,<중립> 저는 상대방의 감정을 바로는 아니지만 어느 정도 알아차릴 수는 있습니다. ...,젠더: 0.\n 나이: 20.\n 질문: 상대방의 감정을 바로 알아차릴 수 있나요?...,0.023676,0.080771,0.034237,0.019078,"[0.00053985947, 0.00074519834, 0.001573527, 0...."
2877,1,30,50,<그렇다> 상대방이 높게 평가할 수 롤 작은 실수도 크게 보일 수 있기 때문이에요....,젠더: 1.\n 나이: 30.\n 질문: 상대방이 자신을 높게 평가하면 나중에 상대...,0.018619,0.065823,0.215068,0.961347,"[0.0045148376, 0.0011999282, 0.0020495104, 0.0..."
2878,1,30,50,<아니다> 상대방이 나를 높게 평가하면 더 잘하면 된다고 생각하기 때문에 상대방이 ...,젠더: 1.\n 나이: 30.\n 질문: 상대방이 자신을 높게 평가하면 나중에 상대...,0.772274,0.020251,0.525608,0.294579,"[0.0004679287, 0.00073236664, 0.00056743925, 0..."


In [ ]:
df_submission_test
batch_size = 200

test_texts_submission=list(df_submission_test['prompt'])
test_labels_submission=list(df_submission_test['Q_number'])

if train_model_type_large: 
  tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-large")
else: 
  tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")

test_dataset_sub = KoreanTextDataset(test_texts_submission, test_labels_submission, tokenizer, max_length=max_length)
test_loader_sub = DataLoader(test_dataset_sub, batch_size=batch_size, shuffle=False)


In [ ]:
model.eval()
model.to(device)
predictions = [[] for i in range(4)]
true_labels = [[] for i in range(4)]

predictions_usrID = []


with torch.no_grad():

    for batch in test_loader_sub:

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)

        # userID predictions
        prob = outputs[4].softmax(dim=1).cpu().numpy() 
        predictions_usrID.append(prob)

        # labels.cpu().numpy()
        for i in range(4):
          
          logits = outputs[i]
          prob = F.sigmoid( logits).cpu().numpy()
          predictions[i].append(prob)
          

print(f"done epoch:  ")


done epoch:  


In [ ]:
for i in range(4):

  np.concatenate(predictions[i])
  name_l = list (MBTI_mapping[i].keys())
  name =  name_l[0] + '/' + name_l[1]

  df_submission_test[name] = np.concatenate(predictions[i])

In [ ]:
ids_predictions = np.concatenate(predictions_usrID)
ids_predictions.shape

(2880, 240)

In [ ]:
# df_submission_test["userIDs predictions"] = ids_predictions

In [ ]:
"idx",

df_sub = df_submission_test[["I/E","S/N","T/F","J/P"]]
df_sub["idx"] = df_submission_test.index+1
df_sub = df_sub[["idx", "I/E","S/N","T/F","J/P"]]
df_sub

<ipython-input-138-037937664a13>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub["idx"] = df_submission_test.index+1


,idx,I/E,S/N,T/F,J/P
0,1,0.288458,0.008788,0.950867,0.003626
1,2,0.977102,0.007157,0.972784,0.991893
2,3,0.951751,0.036827,0.937090,0.049280
3,4,0.987960,0.010907,0.992713,0.025977
4,5,0.003264,0.025398,0.254316,0.986536
...,...,...,...,...,...
2875,2876,0.984188,0.004929,0.555466,0.985570
2876,2877,0.020513,0.038968,0.016395,0.003812
2877,2878,0.004098,0.115473,0.067473,0.974080
2878,2879,0.912116,0.004434,0.656319,0.836956


In [ ]:
df_sub.to_csv( "your path", index = False)

In [ ]:
df_userID_map = train.iloc[::40,][["User_ID", "label", "MBTI"]]
df_userID_map.index = df_userID_map.User_ID
df_userID_dict = dict(df_userID_map["label"])

In [ ]:
# df_userID_dict

In [ ]:
df_submission_test

,Gender,Age,Q_number,Answer,prompt,I/E,S/N,T/F,J/P,userIDs predictions
0,0,30,59,<아니다> 저는 모든 일은 정해진 시간을 지켜서 해야 된다고 생각되어서 마감 기한을...,젠더: 0.\n 나이: 30.\n 질문: 마감 기한을 지키기가 힘든가요? 경험을 이...,0.288458,0.008788,0.950867,0.003626,"[9.7099524e-05, 0.0015523263, 0.00053531304, 0..."
1,1,40,53,<중립> 저는 조용하고 사적인 장소도 좋아하고 사람들로 붐비고 떠들썩한 장소도 좋아...,젠더: 1.\n 나이: 40.\n 질문: 조용하고 사적인 장소보다는 사람들로 붐비고...,0.977102,0.007157,0.972784,0.991893,"[0.0011357091, 0.003327079, 0.0019437977, 0.00..."
2,1,40,56,<그렇다> 저는 규칙을 잘 지키고 매뉴얼 대로 일하는 사람입니다. 그래서 데이터 라...,젠더: 1.\n 나이: 40.\n 질문: 단계를 건너뛰는 일 없이 절차대로 일을 완...,0.951751,0.036827,0.937090,0.049280,"[0.0006127075, 0.0018116662, 0.0009952545, 0.0..."
3,1,40,60,<그렇다> 저는 항상 긍정적인 사고방식을 가지고 살려고 노력하고 있습니다. 이유는 ...,젠더: 1.\n 나이: 40.\n 질문: 일이 원하는 대로 진행될 것이라는 자신감이...,0.987960,0.010907,0.992713,0.025977,"[0.00047402713, 0.000898123, 0.0008986662, 0.0..."
4,1,30,51,<중립> 혼자서 일하는 것도 좋고 함께 일하는 것도 모두 좋은데 같이 의논하는 일도...,젠더: 1.\n 나이: 30.\n 질문: 대부분의 시간을 혼자서 일할 수 있는 직업...,0.003264,0.025398,0.254316,0.986536,"[0.005169144, 0.0013867167, 0.002398192, 0.000..."
...,...,...,...,...,...,...,...,...,...,...
2875,1,20,59,<아니다> 마감 기한은 웬만해서는 지키려고 하는 편이에요. 거의 대부분 지각하지 않아요.,젠더: 1.\n 나이: 20.\n 질문: 마감 기한을 지키기가 힘든가요? 경험을 이...,0.984188,0.004929,0.555466,0.985570,"[0.00061842083, 0.001009285, 0.001005655, 0.00..."
2876,0,20,54,<중립> 저는 상대방의 감정을 바로는 아니지만 어느 정도 알아차릴 수는 있습니다. ...,젠더: 0.\n 나이: 20.\n 질문: 상대방의 감정을 바로 알아차릴 수 있나요?...,0.020513,0.038968,0.016395,0.003812,"[0.00053985947, 0.00074519834, 0.001573527, 0...."
2877,1,30,50,<그렇다> 상대방이 높게 평가할 수 롤 작은 실수도 크게 보일 수 있기 때문이에요....,젠더: 1.\n 나이: 30.\n 질문: 상대방이 자신을 높게 평가하면 나중에 상대...,0.004098,0.115473,0.067473,0.974080,"[0.0045148376, 0.0011999282, 0.0020495104, 0.0..."
2878,1,30,50,<아니다> 상대방이 나를 높게 평가하면 더 잘하면 된다고 생각하기 때문에 상대방이 ...,젠더: 1.\n 나이: 30.\n 질문: 상대방이 자신을 높게 평가하면 나중에 상대...,0.912116,0.004434,0.656319,0.836956,"[0.0004679287, 0.00073236664, 0.00056743925, 0..."


In [ ]:
df_submission_test["userIDs predictions"] = [g for g in ids_predictions]

In [ ]:

df_submission_test.to_pickle("your path")

In [ ]:
def get_MBTI_from_userID (row):
  pred_ID = row["userIDs predictions"]
  ans = 0
  for i in range(len(df_userID_dict)):
    ans += df_userID_dict[i+1][:4]*pred_ID[i]

  return ans
  

In [ ]:
def get_MBTI_from_userID (row):
  pred_ID = row["userIDs predictions"]
  ans = 0
  for i in range(len(df_userID_dict)):
    ans += df_userID_dict[i+1][:4]*pred_ID[i]

  return ans
  
  
mbti_pred_fromUser = df_submission_test.apply(get_MBTI_from_userID, axis = 1).to_numpy()
mbti_pred_fromUser = np.stack(mbti_pred_fromUser)

In [ ]:
df_fromUserIDs = pd.DataFrame(mbti_pred_fromUser, columns = ["I/E","S/N","T/F","J/P"]) 

In [ ]:
df_sub

,idx,I/E,S/N,T/F,J/P
0,1,0.288458,0.008788,0.950867,0.003626
1,2,0.977102,0.007157,0.972784,0.991893
2,3,0.951751,0.036827,0.937090,0.049280
3,4,0.987960,0.010907,0.992713,0.025977
4,5,0.003264,0.025398,0.254316,0.986536
...,...,...,...,...,...
2875,2876,0.984188,0.004929,0.555466,0.985570
2876,2877,0.020513,0.038968,0.016395,0.003812
2877,2878,0.004098,0.115473,0.067473,0.974080
2878,2879,0.912116,0.004434,0.656319,0.836956


In [ ]:
df_fromUserIDs.index = df_sub.idx
df_fromUserIDs


,I/E,S/N,T/F,J/P
idx,,,,
1,0.127696,0.047255,0.879946,0.042288
2,0.779620,0.098296,0.836611,0.830315
3,0.667912,0.232849,0.782777,0.258848
4,0.906563,0.092780,0.907108,0.114384
5,0.082251,0.093003,0.414055,0.918104
...,...,...,...,...
2876,0.920461,0.078620,0.306051,0.929191
2877,0.024793,0.035921,0.022826,0.024183
2878,0.091566,0.110920,0.350354,0.913894


In [ ]:
df_fromUserIDs.to_csv("your path.csv")
# _train_val_combined

In [ ]:
df_fromUserIDs.mean()


I/E    0.557958
S/N    0.479854
T/F    0.556977
J/P    0.490051
dtype: float64

In [ ]:
df_sub_idx = df_sub[["I/E","S/N","T/F","J/P"]]
df_sub_idx.index = df_sub.idx
df_sub_idx

,I/E,S/N,T/F,J/P
idx,,,,
1,0.288458,0.008788,0.950867,0.003626
2,0.977102,0.007157,0.972784,0.991893
3,0.951751,0.036827,0.937090,0.049280
4,0.987960,0.010907,0.992713,0.025977
5,0.003264,0.025398,0.254316,0.986536
...,...,...,...,...
2876,0.984188,0.004929,0.555466,0.985570
2877,0.020513,0.038968,0.016395,0.003812
2878,0.004098,0.115473,0.067473,0.974080


In [ ]:
df_fromUserIDs

,I/E,S/N,T/F,J/P
idx,,,,
1,0.127696,0.047255,0.879946,0.042288
2,0.779620,0.098296,0.836611,0.830315
3,0.667912,0.232849,0.782777,0.258848
4,0.906563,0.092780,0.907108,0.114384
5,0.082251,0.093003,0.414055,0.918104
...,...,...,...,...
2876,0.920461,0.078620,0.306051,0.929191
2877,0.024793,0.035921,0.022826,0.024183
2878,0.091566,0.110920,0.350354,0.913894


In [ ]:
df_sub_idx

,I/E,S/N,T/F,J/P
idx,,,,
1,0.288458,0.008788,0.950867,0.003626
2,0.977102,0.007157,0.972784,0.991893
3,0.951751,0.036827,0.937090,0.049280
4,0.987960,0.010907,0.992713,0.025977
5,0.003264,0.025398,0.254316,0.986536
...,...,...,...,...
2876,0.984188,0.004929,0.555466,0.985570
2877,0.020513,0.038968,0.016395,0.003812
2878,0.004098,0.115473,0.067473,0.974080


In [ ]:
df_new = (df_sub_idx+df_fromUserIDs)/2
df_new

,I/E,S/N,T/F,J/P
idx,,,,
1,0.208077,0.028021,0.915406,0.022957
2,0.878361,0.052726,0.904698,0.911104
3,0.809831,0.134838,0.859934,0.154064
4,0.947262,0.051843,0.949910,0.070180
5,0.042758,0.059201,0.334185,0.952320
...,...,...,...,...
2876,0.952324,0.041775,0.430759,0.957380
2877,0.022653,0.037444,0.019611,0.013998
2878,0.047832,0.113197,0.208914,0.943987


In [ ]:
df_new.mean()

I/E    0.581386
S/N    0.468242
T/F    0.579229
J/P    0.489402
dtype: float64

In [ ]:
df_new.to_csv("your path")

# experiment with sub file

In [ ]:
df_sub_res = pd.read_pickle("your path")
df_sub_res.head()

,Gender,Age,Q_number,Answer,prompt,I/E,S/N,T/F,J/P,userIDs predictions
0,0,30,59,<아니다> 저는 모든 일은 정해진 시간을 지켜서 해야 된다고 생각되어서 마감 기한을...,젠더: 0.\n 나이: 30.\n 질문: 마감 기한을 지키기가 힘든가요? 경험을 이...,0.072754,0.059611,0.807116,0.002243,"[0.000112223126, 0.0010957714, 0.00022895438, ..."
1,1,40,53,<중립> 저는 조용하고 사적인 장소도 좋아하고 사람들로 붐비고 떠들썩한 장소도 좋아...,젠더: 1.\n 나이: 40.\n 질문: 조용하고 사적인 장소보다는 사람들로 붐비고...,0.210719,0.049920,0.995801,0.992366,"[0.001346076, 0.0024603358, 0.0008750284, 0.00..."
2,1,40,56,<그렇다> 저는 규칙을 잘 지키고 매뉴얼 대로 일하는 사람입니다. 그래서 데이터 라...,젠더: 1.\n 나이: 40.\n 질문: 단계를 건너뛰는 일 없이 절차대로 일을 완...,0.006600,0.966651,0.988632,0.167495,"[0.0020937566, 0.0009185381, 0.00047728428, 0...."
3,1,40,60,<그렇다> 저는 항상 긍정적인 사고방식을 가지고 살려고 노력하고 있습니다. 이유는 ...,젠더: 1.\n 나이: 40.\n 질문: 일이 원하는 대로 진행될 것이라는 자신감이...,0.910936,0.340525,0.962694,0.020533,"[0.00039621769, 0.0010459482, 0.00044874955, 0..."
4,1,30,51,<중립> 혼자서 일하는 것도 좋고 함께 일하는 것도 모두 좋은데 같이 의논하는 일도...,젠더: 1.\n 나이: 30.\n 질문: 대부분의 시간을 혼자서 일할 수 있는 직업...,0.008071,0.047633,0.080825,0.991751,"[0.003009796, 0.00045923458, 0.0027498526, 0.0..."


In [ ]:
df_userID_map = train.iloc[::40,][["User_ID", "label", "MBTI", "Gender", "Age"]]
df_userID_map.index = df_userID_map.User_ID
df_userID_dict = dict(df_userID_map["label"])
df_userID_dict_Age = dict(df_userID_map["Age"])
df_userID_dict_Gender = dict(df_userID_map["Gender"])

def get_MBTI_from_userID (row):
  pred_ID = row["userIDs predictions"]
  ans = 0
  iters = 0
  for i in range(len(df_userID_dict)):
    if df_userID_dict_Age[i+1] == row.Age:
      if df_userID_dict_Gender[i+1] == row.Gender:
        iters+=1
        ans += df_userID_dict[i+1][:4]*pred_ID[i]
  return ans


In [ ]:
mbti_pred_fromUser = df_sub_res.apply(get_MBTI_from_userID, axis = 1).to_numpy()
mbti_pred_fromUser = np.stack(mbti_pred_fromUser)
df_sub_normal = pd.DataFrame(mbti_pred_fromUser, columns = ["I/E","S/N","T/F","J/P"])
df_sub_normal["Gender"]=df_sub_res["Gender"]
df_sub_normal["Age"]=df_sub_res["Age"]


In [ ]:
idx_index = np.array(list(range(1, 2881))) 
mbti_columns = ["I/E","S/N","T/F","J/P"]
idx_index

array([   1,    2,    3, ..., 2878, 2879, 2880])

In [ ]:
!ls

 data
 df_submission_test_model1.csv
 diyar
'diyar test.ipynb'
'Diyar training.ipynb'
 HackathonDataAnalysis.ipynb
'Korean Api mine strachs.ipynb'
 models
 results
 sub_clean2.csv
 submission_large_model_1_train_val_combined.csv
 zhantai


In [ ]:
df_sub["idx"] = idx_index

<ipython-input-85-010cf7a29d1f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub["idx"] = idx_index


In [ ]:
df_sub[ ["idx"] + mbti_columns ].to_csv("sub_clean2.csv", index = False)

In [ ]:
mbti_columns = ["I/E","S/N","T/F","J/P"]
df_sub_normal

,I/E,S/N,T/F,J/P,Gender,Age
0,0.108524,0.033152,0.745550,0.019269,0,30
1,0.232301,0.067199,0.809284,0.710928,1,40
2,0.100343,0.615747,0.718867,0.185377,1,40
3,0.709784,0.319441,0.738460,0.080535,1,40
4,0.069203,0.087219,0.208258,0.824364,1,30
...,...,...,...,...,...,...
2875,0.854253,0.099387,0.469805,0.871030,1,20
2876,0.010140,0.053953,0.010682,0.009126,0,20
2877,0.058195,0.051967,0.238708,0.810119,1,30
2878,0.766782,0.041644,0.652651,0.343974,1,30


In [ ]:
df_new = df_sub_normal.copy()

In [ ]:
Age_vals = [20, 30, 40]
mask = df_new.Age == Age_vals[1]

In [ ]:
df_new[mask][mbti_columns]

,I/E,S/N,T/F,J/P
0,0.108524,0.033152,0.745550,0.019269
4,0.069203,0.087219,0.208258,0.824364
9,0.717244,0.295645,0.675701,0.393079
10,0.612735,0.182583,0.329221,0.394286
12,0.171393,0.451629,0.665682,0.653902
...,...,...,...,...
2871,0.116598,0.286520,0.535656,0.803303
2872,0.120696,0.388214,0.366302,0.406586
2874,0.179866,0.814344,0.286450,0.623931
2877,0.058195,0.051967,0.238708,0.810119


In [ ]:
for col in mbti_columns:
  print (df_sub_normal[col])
  break

0       0.108524
1       0.232301
2       0.100343
3       0.709784
4       0.069203
          ...   
2875    0.854253
2876    0.010140
2877    0.058195
2878    0.766782
2879    0.815113
Name: I/E, Length: 2880, dtype: float64


In [ ]:
m = df_sub_normal[col].mean()
m

0.44422098157729123

In [ ]:
a = df_sub_normal[col]*(0.5/m)
a.mean()

0.5

0.006317694213611927

In [ ]:
df_sub_normal.groupby("Age").mean()

,I/E,S/N,T/F,J/P,Gender
Age,,,,,
20,0.630302,0.446675,0.396349,0.496283,0.657895
30,0.476653,0.400860,0.423730,0.459330,0.610000
40,0.343101,0.440243,0.529912,0.349979,0.598039


In [ ]:
df_sub_normal.groupby("Gender").mean()

,I/E,S/N,T/F,J/P,Age
Gender,,,,,
0,0.347272,0.506541,0.426864,0.390635,33.010753
1,0.505556,0.373171,0.488346,0.436466,32.448980


# saved model

In [ ]:
model = MBTI_UserID_Model(large = train_model_type_large, dropout_per = 0.5 )

model_name = "large_3_"
epoch = "final"
PATH = "your path"

model.load_state_dict(torch.load(PATH))
model.eval()

Some weights of the model checkpoint at beomi/kcbert-large were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


MBTI_UserID_Model(
  (base_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 1024, padding_idx=0)
      (position_embeddings): Embedding(300, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps

# create phase 2 datasets

In [ ]:
df_train = pd.read_excel("train_data.xlsx")
df_test = pd.read_excel("train_data.xlsx")

In [ ]:
df_train.head()

,Data_ID,User_ID,Gender,Age,MBTI,Q_number,Short_Answer,Long_Answer
0,481,9,female,30,INTJ,1,아니다,새로운 사람을 만나서 이야기를 나누고 마음이 통하는 친구가 되기까지의 과정은 많은 ...
1,482,9,female,30,INTJ,2,그렇다,자유시간이 생기면 평소에 관심을 두고 있던 분야에 대해 공부하고 탐구하는 시간을 주...
2,483,9,female,30,INTJ,3,그렇다,다른 사람이 울고 있는 모습만 볼 때는 왜 울고 있는지 이유를 몰라 공감 되지 않는...
3,484,9,female,30,INTJ,4,그렇다,모든 일이 마음먹은 대로 진행되는 것은 아니기에 미리 여러 가지 대책을 세워둬야 계...
4,485,9,female,30,INTJ,5,그렇다,퇴사 면담을 하면서 대외적인 이유가 아닌 진짜 퇴사의 이유를 솔직하게 말해 달라는 ...


In [ ]:
train.head()

,Data_ID,User_ID,Gender,Age,MBTI,Q_number,Answer,prompt,label
0,1,1,1,30,INFP,1,<아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...,젠더: 1.\n 나이: 30.\n 질문: 주기적으로 새로운 친구를 만드나요? 경험을...,"[0, 1, 1, 1, 0]"
1,2,1,1,30,INFP,2,<중립> 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하...,젠더: 1.\n 나이: 30.\n 질문: 자유 시간 중 상당 부분을 다양한 관심사를...,"[0, 1, 1, 1, 0]"
2,3,1,1,30,INFP,3,<그렇다> 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ...,젠더: 1.\n 나이: 30.\n 질문: 다른 사람이 울고 있는 모습을 보면 자신도...,"[0, 1, 1, 1, 0]"
3,4,1,1,30,INFP,4,<중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고...,젠더: 1.\n 나이: 30.\n 질문: 일이 잘못될 때를 대비해 여러 대비책을 세...,"[0, 1, 1, 1, 0]"
4,5,1,1,30,INFP,5,<아니다> 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대...,젠더: 1.\n 나이: 30.\n 질문: 압박감이 심한 환경에서도 평정심을 유지하는...,"[0, 1, 1, 1, 0]"


In [ ]:
# MBTI mapping 
MBTI_mapping = [
    {'I': 0, 'E': 1},
    {'S': 0, 'N': 1},
    {'T': 0, 'F': 1},
    {'J': 0, 'P': 1},
]
# 0 - ISTJ; 1 - ENFP 


In [ ]:
def create_label_mbti (mbti):
  return np.array([MBTI_mapping[i][letter] for i, letter in enumerate(mbti)])

print ("INFP: ", create_label_mbti("INFP"))


def create_label_user_ID (user_ID):
  b = np.zeros([240])
  b[user_ID-1]=1
  return b

INFP:  [0 1 1 1]


In [ ]:
def create_promt (row):
    p = f"젠더: {row.Gender}.\n 나이: {row.Age}.\n 질문: {df_questions.Question.loc[row.Q_number-1]}.\n 정답: <{row.Short_Answer}> {row.Long_Answer}.\n"
    return p


In [ ]:
df_train["prompt"] = df_train.apply(create_promt, axis=1)
df_test["prompt"] = df_test.apply(create_promt, axis=1)


In [ ]:
# IMPORTANT: ID's the model will predict are from 0 to 239
def create_label (row):
  mbti = create_label_mbti(row.MBTI)
  id = row.User_ID-1 
  return np.concatenate( [mbti, np.array([id]) ] , axis = 0)

df_train["label"] = df_train.apply(create_label, axis=1)



# eval phase 1 model on phase 2 data

In [ ]:
df_train

score for 0: 0.7786230468749998. 
score for 1: 0.7391612413194445. 
score for 2: 0.7508116319444444. 
score for 3: 0.7530783420138889. 
accuracy: 0.2875
losses: 2.765681266784668 2.8310184478759766
